## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [0]:
import os
from tqdm import tqdm 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten,BatchNormalization
import matplotlib.pyplot as plt 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
project_path = "/content/drive/My Drive/DogBreed_Classification"

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [7]:
from zipfile import ZipFile
with ZipFile(project_path+'/train.zip', 'r') as z:
  z.extractall('/content/drive/My Drive/DogTrain')

OSError: ignored

Repeat the same step for test.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'/test.zip', 'r') as z:
  z.extractall('/content/drive/My Drive/DogTrain')

Repeat the same step for sample_submission.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile('/content/drive/My Drive/DogBreed_Classification/sample_submission.csv.zip', 'r') as z:
  z.extractall('/content/drive/My Drive/DogTrain')

Repeat the same step for labels.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile('/content/drive/My Drive/DogBreed_Classification/labels.csv.zip', 'r') as z:
  z.extractall('/content/drive/My Drive/DogTrain')

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [0]:
df_train = pd.read_csv('/content/drive/My Drive/DogTrain/labels.csv')

In [0]:
df_test = pd.read_csv('/content/drive/My Drive/DogTrain/sample_submission.csv')


In [12]:
df_train.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


### Print the count of each category of Dogs given in the dataset



In [16]:
df_train['breed'].value_counts()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
                       ... 
brabancon_griffon        67
golden_retriever         67
komondor                 67
briard                   66
eskimo_dog               66
Name: breed, Length: 120, dtype: int64

### Get one-hot encodings of labels

In [0]:
targets_series = pd.Series(df_train['breed']) 


In [0]:
one_hot = pd.get_dummies(targets_series, sparse = True) 


In [0]:
one_hot_labels = np.asarray(one_hot)


In [27]:
one_hot_labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [0]:
im_size = 128

x_train = []
y_train = []
x_test = []

In [29]:
i = 0 
for f, breed in tqdm(df_train.values): #tqdm은 반복문의 진행상황을 알려준다.
    img = cv2.imread('/content/drive/My Drive/DogTrain/train/{}.jpg'.format(f))
    label = one_hot_labels[i]
    x_train.append(cv2.resize(img, (im_size, im_size)))
    y_train.append(label)
    i += 1

100%|██████████| 10222/10222 [01:16<00:00, 132.83it/s]


In [38]:
print(np.asarray(x_train).shape)
print(np.asarray(y_train).shape)

(10222, 128, 128, 3)
(10222, 120)


In [33]:
for f in tqdm(df_test['id'].values):
    img = cv2.imread('/content/drive/My Drive/DogTrain/test/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (im_size, im_size)))

100%|██████████| 10357/10357 [01:13<00:00, 141.22it/s]


Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [0]:
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.
x_test  = np.array(x_test, np.float32) / 255. 

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [40]:
print(x_train_raw.shape)
print(y_train_raw.shape)
print(x_test.shape)

(10222, 128, 128, 3)
(10222, 120)
(10357, 128, 128, 3)


In [0]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.3, random_state=1)


In [56]:
print(X_train.shape)
print(X_valid.shape)

(7155, 128, 128, 3)
(3067, 128, 128, 3)


### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [0]:
#This step was done above

Run the below code to load the test image files in x_test_feature

In [0]:
#This has been done abve

Normalize the test data and convert it into 4 dimensions

In [0]:
#This was done above

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [59]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dense
from keras.optimizers import Adam
# initialize the model

inputShape = (128, 128, 3)
EPOCHS = 10
INIT_LR = 1e-3
BS = 128

model = Sequential()
model.add(Conv2D(32, (5, 5), padding="same", input_shape=inputShape)) 
model.add(Activation("relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(units=256))
model.add(Activation("relu"))

# softmax classifier
model.add(Dense(units=120))
model.add(Activation("softmax"))
   
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 128, 128, 32)      2432      
_________________________________________________________________
activation_5 (Activation)    (None, 128, 128, 32)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
activation_6 (Activation)    (None, 64, 64, 64)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 65536)            

### Use batch_size = 128 and epochs = 10 and execute the model

In [45]:

#Training on the dataset
model.fit(X_train, Y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(X_valid, Y_valid))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 7155 samples, validate on 3067 samples
Epoch 1/10





7155/7155 [==============================] - 19s 3ms/step - loss: 4.9703 - acc: 0.0105 - val_loss: 4.7900 - val_acc: 0.0104
Epoch 2/10
7155/7155 [==============================] - 3s 487us/step - loss: 4.7488 - acc: 0.0198 - val_loss: 4.7331 - val_acc: 0.0192
Epoch 3/10
7155/7155 [==============================] - 3s 481us/step - loss: 4.4799 - acc: 0.0514 - val_loss: 4.6229 - val_acc: 0.0333
Epoch 4/10
7155/7155 [==============================] - 4s 492us/step - loss: 3.6432 - acc: 0.1823 - val_loss: 4.8556 - val_acc: 0.0333
Epoch 5/10
7155/7155 [==============================] - 3s 489us/step - loss: 2.2929 - acc: 0.4618 - val_loss: 5.8989 - val_acc: 0.0372
Epoch 6/10
7155/7155 [==============================] - 4s 490us/step - loss: 1.0607 - acc: 0.7507 - val_loss: 7.5867 - val_acc: 0.0352
Epoch 7/10
7155/7155 [=============

In [48]:
loss,accuracy=model.evaluate(X_valid,Y_valid)

3067/3067 [==============================] - 1s 209us/step


In [49]:
accuracy

0.03521356375035922

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [0]:
from keras.preprocessing.image import ImageDataGenerator

traingen = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, 
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
   horizontal_flip=True, fill_mode="nearest")
valgen = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, 
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
   horizontal_flip=True, fill_mode="nearest")

In [53]:

H = model.fit_generator(aug.flow(X_train, Y_train, batch_size=BS), 
                        validation_data=(X_valid, Y_valid), 
                        steps_per_epoch=len(x_train) // BS, 
                        epochs=10, verbose=1)

Epoch 1/10
79/79 [==============================] - 36s 460ms/step - loss: 5.0676 - acc: 0.0147 - val_loss: 4.7740 - val_acc: 0.0150
Epoch 2/10
79/79 [==============================] - 35s 444ms/step - loss: 4.7482 - acc: 0.0174 - val_loss: 4.7540 - val_acc: 0.0209
Epoch 3/10
79/79 [==============================] - 36s 449ms/step - loss: 4.6900 - acc: 0.0234 - val_loss: 4.6946 - val_acc: 0.0313
Epoch 4/10
79/79 [==============================] - 35s 441ms/step - loss: 4.6124 - acc: 0.0290 - val_loss: 4.6379 - val_acc: 0.0241
Epoch 5/10
79/79 [==============================] - 35s 447ms/step - loss: 4.5371 - acc: 0.0399 - val_loss: 4.5747 - val_acc: 0.0398
Epoch 6/10
79/79 [==============================] - 35s 445ms/step - loss: 4.4612 - acc: 0.0444 - val_loss: 4.5629 - val_acc: 0.0381
Epoch 7/10
79/79 [==============================] - 34s 437ms/step - loss: 4.3961 - acc: 0.0483 - val_loss: 4.4872 - val_acc: 0.0450
Epoch 8/10
79/79 [==============================] - 36s 451ms/step - 

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [0]:
train_generator=traingen.flow(X_train, Y_train, batch_size=BS)
val_generator=valgen.flow(X_valid, Y_valid, batch_size=BS)


### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [60]:

H = model.fit_generator(train_generator, 
                        validation_data=val_generator, 
                        steps_per_epoch=100, 
                        epochs=10,validation_steps=50, verbose=1
                        )

Epoch 1/10
100/100 [==============================] - 69s 690ms/step - loss: 4.8256 - acc: 0.0118 - val_loss: 4.7220 - val_acc: 0.0147
Epoch 2/10
100/100 [==============================] - 63s 632ms/step - loss: 4.6001 - acc: 0.0290 - val_loss: 4.5529 - val_acc: 0.0310
Epoch 3/10
100/100 [==============================] - 65s 648ms/step - loss: 4.3827 - acc: 0.0466 - val_loss: 4.4242 - val_acc: 0.0413
Epoch 4/10
100/100 [==============================] - 65s 649ms/step - loss: 4.2716 - acc: 0.0617 - val_loss: 4.3542 - val_acc: 0.0507
Epoch 5/10
100/100 [==============================] - 63s 634ms/step - loss: 4.1835 - acc: 0.0706 - val_loss: 4.3313 - val_acc: 0.0515
Epoch 6/10
100/100 [==============================] - 67s 673ms/step - loss: 4.1046 - acc: 0.0823 - val_loss: 4.3923 - val_acc: 0.0562
Epoch 7/10
100/100 [==============================] - 64s 642ms/step - loss: 4.0392 - acc: 0.0936 - val_loss: 4.3072 - val_acc: 0.0643
Epoch 8/10
100/100 [==============================] - 6

# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [0]:
from keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=('/content/vgg16_weights_tf_dim_ordering_tf_kernels_notop (1).h5'),
                 include_top=False, pooling='avg')

Print the summary of the base_model

In [73]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [0]:
x = base_model.output
x = Flatten()(x)
predictions = Dense(120, activation='softmax')(x)

In [83]:
my_new_model = Sequential()
my_new_model.add((VGG16(weights=('/content/vgg16_weights_tf_dim_ordering_tf_kernels_notop (1).h5'),
                 include_top=False, pooling='avg')))
my_new_model.add(Dense(units=1024))
my_new_model.add(Activation('relu'))
my_new_model.add(Dropout(0.5))
my_new_model.add(Dense(256))
my_new_model.add(Activation('relu'))
my_new_model.add(Dropout(0.5))
my_new_model.add(Dense(120, activation='softmax'))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Make all the layers in the base_model (VGG16) to be non-trainable

In [0]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
my_new_model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [84]:
my_new_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              525312    
_________________________________________________________________
activation_9 (Activation)    (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               262400    
_________________________________________________________________
activation_10 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)              

In [0]:
for layer in my_new_model.layers:
  if('dense' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

In [87]:

#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in my_new_model.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

vgg16
False
dense_5
True
activation_9
False
dropout_1
False
dense_6
True
activation_10
False
dropout_2
False
dense_7
True


### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [91]:
my_new_model.fit(X_train, Y_train,
          batch_size=128,
          epochs=50,
          verbose=1,
          validation_data=(X_valid, Y_valid))

Train on 7155 samples, validate on 3067 samples
Epoch 1/50
7155/7155 [==============================] - 8s 1ms/step - loss: 3.5007 - acc: 0.1588 - val_loss: 3.4074 - val_acc: 0.1842
Epoch 2/50
7155/7155 [==============================] - 8s 1ms/step - loss: 3.4316 - acc: 0.1725 - val_loss: 3.3860 - val_acc: 0.1894
Epoch 3/50
7155/7155 [==============================] - 8s 1ms/step - loss: 3.3584 - acc: 0.1744 - val_loss: 3.3238 - val_acc: 0.1940
Epoch 4/50
7155/7155 [==============================] - 8s 1ms/step - loss: 3.2945 - acc: 0.1901 - val_loss: 3.3020 - val_acc: 0.1943
Epoch 5/50
7155/7155 [==============================] - 8s 1ms/step - loss: 3.2300 - acc: 0.2015 - val_loss: 3.2789 - val_acc: 0.1982
Epoch 6/50
7155/7155 [==============================] - 8s 1ms/step - loss: 3.1747 - acc: 0.2122 - val_loss: 3.2248 - val_acc: 0.2080
Epoch 7/50
7155/7155 [==============================] - 8s 1ms/step - loss: 3.1179 - acc: 0.2217 - val_loss: 3.2374 - val_acc: 0.2067
Epoch 8/50
715

In [0]:
#Optimising the model using BatchNorm and DropOut

In [0]:
my_new_model = Sequential()
my_new_model.add((VGG16(weights=('/content/vgg16_weights_tf_dim_ordering_tf_kernels_notop (1).h5'),
                 include_top=False, pooling='avg')))
my_new_model.add(Dense(units=1024))
my_new_model.add(BatchNormalization())
my_new_model.add(Activation('relu'))
my_new_model.add(Dropout(0.5))
my_new_model.add(Dense(256))
my_new_model.add(BatchNormalization())
my_new_model.add(Activation('relu'))
my_new_model.add(Dropout(0.5))
my_new_model.add(Dense(120, activation='softmax'))

In [96]:
my_new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_9 (Dense)              (None, 1024)              525312    
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
activation_11 (Activation)   (None, 1024)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)               262400    
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)              

In [0]:
for layer in my_new_model.layers:
  if('dense' not in layer.name and 'batch' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

In [98]:
for layer in my_new_model.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

vgg16
False
dense_9
True
batch_normalization_1
True
activation_11
False
dropout_3
False
dense_10
True
batch_normalization_2
True
activation_12
False
dropout_4
False
dense_11
True


In [0]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
my_new_model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [101]:
my_new_model.fit(X_train, Y_train,
          batch_size=128,
          epochs=50,
          verbose=1,
          validation_data=(X_valid, Y_valid))

Train on 7155 samples, validate on 3067 samples
Epoch 1/50
7155/7155 [==============================] - 9s 1ms/step - loss: 5.0087 - acc: 0.0238 - val_loss: 4.3558 - val_acc: 0.0717
Epoch 2/50
7155/7155 [==============================] - 8s 1ms/step - loss: 4.3352 - acc: 0.0801 - val_loss: 4.0210 - val_acc: 0.1171
Epoch 3/50
7155/7155 [==============================] - 8s 1ms/step - loss: 3.9253 - acc: 0.1276 - val_loss: 3.8206 - val_acc: 0.1477
Epoch 4/50
7155/7155 [==============================] - 8s 1ms/step - loss: 3.6205 - acc: 0.1739 - val_loss: 3.6770 - val_acc: 0.1722
Epoch 5/50
7155/7155 [==============================] - 8s 1ms/step - loss: 3.4158 - acc: 0.2048 - val_loss: 3.5909 - val_acc: 0.1881
Epoch 6/50
7155/7155 [==============================] - 8s 1ms/step - loss: 3.2262 - acc: 0.2331 - val_loss: 3.4634 - val_acc: 0.2028
Epoch 7/50
7155/7155 [==============================] - 8s 1ms/step - loss: 3.0413 - acc: 0.2615 - val_loss: 3.4468 - val_acc: 0.1940
Epoch 8/50
715